In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Check for GPU availability
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("GPU is available and will be used for training.")
else:
    print("GPU is not available, using CPU.")

# Load and preprocess the dataset
dataset_path = "./dataset/blackhole.csv"  # Replace with your dataset path
data = pd.read_csv(dataset_path)

# Preprocessing
features = data.drop(columns=['category', 'label'])
labels = data['label']

# Normalize features
scaler = StandardScaler()
features = scaler.fit_transform(features)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Further split training set into client datasets
num_clients = 5
client_data = np.array_split(X_train, num_clients)
client_labels = np.array_split(y_train, num_clients)

# Define a more complex neural network
def create_model(input_shape):
    model = Sequential([
        InputLayer(shape=(input_shape,)),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(32, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Initialize a global model
global_model = create_model(X_train.shape[1])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


GPU is available and will be used for training.


/home/ebinalex/miniconda3/envs/tf/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)
I0000 00:00:1722001544.658811   54311 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1722001544.658894   54311 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1722001544.658917   54311 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1722001546.001234   54311 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel m

Federated Learning and Few-Shot Learning Implementation

In [7]:
# Federated learning simulation with fine-tuning
def federated_training(global_model, client_data, client_labels, epochs=50):
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        client_weights = []
        for i in range(num_clients):
            # Clone the global model
            local_model = tf.keras.models.clone_model(global_model)
            local_model.set_weights(global_model.get_weights())
            
            # Compile the cloned model
            local_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
            
            # Train on client's data with validation and early stopping
            local_model.fit(client_data[i], client_labels[i], epochs=1, batch_size=32, verbose=0, validation_split=0.1, callbacks=[early_stopping])
            
            # Collect the weights after training
            client_weights.append(local_model.get_weights())
        
        # Average the weights to update the global model
        new_weights = [np.mean([client_weight[layer] for client_weight in client_weights], axis=0) for layer in range(len(client_weights[0]))]
        global_model.set_weights(new_weights)

# Train the global model with federated learning
federated_training(global_model, client_data, client_labels, epochs=50)

# Evaluate on the test set
y_pred = (global_model.predict(X_test) > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

Epoch 1/50


I0000 00:00:1722001554.228786   54958 service.cc:146] XLA service 0x7f8f70073b40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1722001554.228833   54958 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2024-07-26 19:15:54.256968: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-26 19:15:54.467901: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
I0000 00:00:1722001556.056018   54958 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Epoch 2/50
Epoch 3/50
Epoch 4/50
Epoch 5/50
Epoch 6/50
Epoch 7/50
Epoch 8/50
Epoch 9/50
Epoch 10/50


In [2]:
import tensorflow as tf

# Check if a GPU is available
if tf.config.list_physical_devices('GPU'):
    print("GPU is available and will be used for training.")
else:
    print("GPU is not available, using CPU.")


2024-07-26 19:13:16.677966: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-26 19:13:16.690726: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-26 19:13:16.694461: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-26 19:13:16.704345: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-26 19:13:17.423734: W tensorflow/compiler/tf2

GPU is available and will be used for training.


I0000 00:00:1722001399.142289   54311 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1722001399.160897   54311 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1722001399.160939   54311 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))



Num GPUs Available:  1


In [4]:
import torch
print(torch.cuda.is_available()) # should be True

t = torch.rand(10, 10).cuda()
print(t.device) # should be CUDA

ModuleNotFoundError: No module named 'torch'